# MLP

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

import pandas as pd
import numpy as np
from datetime import datetime

from scripts.constants import X_TRAIN
from scripts.constants import X_TEST
from scripts.constants import Y_TRAIN
from scripts.constants import Y_TEST

from scripts.data import scaffold_dataloaders

from scripts.training import train_test_loop

2024-02-18 12:51:32.422068: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_dataloader, test_dataloader = scaffold_dataloaders(X_TRAIN, X_TEST, Y_TRAIN, Y_TEST)

In [4]:
x, y = next(iter(train_dataloader))
print(x)
print(y)

tensor([[ 1.2051,  1.2021,  1.1673,  ...,  0.9636,  0.9794,  0.9781],
        [ 1.0533,  1.0573,  1.0304,  ...,  0.7042,  0.7231,  0.7181],
        [-0.0200, -0.0700, -0.1400,  ..., -0.3122, -0.2807, -0.3011],
        ...,
        [ 0.3804,  0.3328,  0.2615,  ...,  0.2941,  0.3447,  0.3615],
        [-0.4490, -0.4413, -0.4667,  ..., -0.5019, -0.4541, -0.4507],
        [-0.5691, -0.5921, -0.6341,  ..., -0.4179, -0.4033, -0.4432]])
tensor([[3],
        [1],
        [2],
        [2],
        [0],
        [3],
        [0],
        [0]])


In [5]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()

        self.dense = nn.Linear(1440, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.dense(x)
        x = self.softmax(x)
        return x

## Using SGD

In [6]:
model = MLP()
model

MLP(
  (dense): Linear(in_features=1440, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

In [7]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/mlp_sgd_{}'.format(timestamp))

In [9]:
train_test_loop("mlp_sgd", model, writer, train_dataloader, test_dataloader, optimizer, loss_fn)

EPOCH 0:
  batch 1 loss: 1.3572397232055664
  batch 2 loss: 1.438297986984253
  batch 3 loss: 1.3376415967941284
  batch 4 loss: 1.4353207349777222
  batch 5 loss: 1.3565753698349
  batch 6 loss: 1.4245274066925049
  batch 7 loss: 1.4130030870437622
  batch 8 loss: 1.40128493309021
  batch 9 loss: 1.427922248840332
  batch 10 loss: 1.4664177894592285
  batch 11 loss: 1.4026010036468506
  batch 12 loss: 1.3651753664016724
  batch 13 loss: 1.3559849262237549
  batch 14 loss: 1.404484748840332
  batch 15 loss: 1.415113925933838
LOSS train 1.415113925933838 valid 1.371384620666504
ACCURACY train: 0.375 valid 0.5
EPOCH 1:
  batch 1 loss: 1.349901556968689
  batch 2 loss: 1.405714988708496
  batch 3 loss: 1.3836859464645386
  batch 4 loss: 1.297317385673523
  batch 5 loss: 1.3329689502716064
  batch 6 loss: 1.4291081428527832
  batch 7 loss: 1.3856496810913086
  batch 8 loss: 1.3550256490707397
  batch 9 loss: 1.3006168603897095
  batch 10 loss: 1.3847801685333252
  batch 11 loss: 1.29840254

## Using Adam

In [10]:
model = MLP()
model

MLP(
  (dense): Linear(in_features=1440, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

In [11]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [12]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/mlp_adam_{}'.format(timestamp))

In [13]:
train_test_loop("mlp_adam", model, writer, train_dataloader, test_dataloader, optimizer, loss_fn)

EPOCH 0:
  batch 1 loss: 1.4202218055725098
  batch 2 loss: 1.4354478120803833
  batch 3 loss: 1.2075896263122559
  batch 4 loss: 1.2415415048599243
  batch 5 loss: 1.4462474584579468
  batch 6 loss: 1.403373122215271
  batch 7 loss: 1.4433950185775757
  batch 8 loss: 1.4856009483337402
  batch 9 loss: 1.4312715530395508
  batch 10 loss: 1.5799949169158936
  batch 11 loss: 1.3574556112289429
  batch 12 loss: 1.516857385635376
  batch 13 loss: 1.500300407409668
  batch 14 loss: 1.5955536365509033
  batch 15 loss: 1.5199995040893555
LOSS train 1.5199995040893555 valid 1.5118473768234253
ACCURACY train: 0.125 valid 0.0
EPOCH 1:
  batch 1 loss: 1.207152009010315
  batch 2 loss: 1.2318730354309082
  batch 3 loss: 1.3591103553771973
  batch 4 loss: 1.4807466268539429
  batch 5 loss: 1.2847745418548584
  batch 6 loss: 1.497134804725647
  batch 7 loss: 1.3608430624008179
  batch 8 loss: 1.5404229164123535
  batch 9 loss: 1.3645076751708984
  batch 10 loss: 1.4598866701126099
  batch 11 loss: 1